# Хакатон AI.Hack Москва.
## Газпром Нефть

https://www.kaggle.com/c/ai-hack-2018-msk-gpn/leaderboard

In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
# предопределим типы колонок для экономии памяти
dtypes = {
    'ID': np.int32,
    'CLIENT': np.int32,
    'DATA_TRANS': str,
    'AZS_NUMBER': np.int16,
    'REGION_AZS': np.int16,
    'VID_NP': np.int16,
    'COL_LITR':  np.float32,
    'CENA_CLIENT': np.float32
    }

In [3]:
train = pd.read_csv('datasets/gpn_b2b/train_data.csv', dtype = dtypes)

In [4]:
test = pd.read_csv('datasets/gpn_b2b/test_data.csv', dtype = dtypes)

In [5]:
def month(s): #для обучения сезонности
    return int(s[3:5])

def year(s):
    return int(s[8:10])

def date(s): #для пристегивания фич
    return s[3:10]

In [6]:
%%time

train['month'] = train['DATA_TRANS'].apply(month).astype(float)

test['month'] = test['DATA_TRANS'].apply(month).astype(float)

CPU times: user 24.8 s, sys: 927 ms, total: 25.7 s
Wall time: 25.7 s


In [7]:
train_sel = train  # catboost не хватало оперативы и использовал только часть, удалено, т.к. lightgbm показал себя на этих данных лучше

In [8]:
%%time

features = pd.read_csv('datasets/gpn_b2b/features_monthly.csv') # сгенерировано с полным тестом в features_monthly.ipynb

trans_count_d = {} # немножко покешируем т.к. вытаскивание из датафрейма медленнее
ltr_mean_d = {}

for index, row in features.iterrows():
    trans_count_d[row['client_date']] = row['trans_count']
    ltr_mean_d[row['client_date']] = row['ltr_mean']

def trans_count(c):
    try:
        return trans_count_d[c]
    except:
        return np.nan

def ltr_mean(c):
    try:
        return ltr_mean_d[c]
    except:
        return np.nan

CPU times: user 48.7 s, sys: 47.3 ms, total: 48.8 s
Wall time: 48.8 s


In [9]:
%%time

train_sel['date'] = train_sel['DATA_TRANS'].apply(date)
train_sel['client_date'] = train_sel['CLIENT'].astype(str) + '_' + train_sel['date'].astype(str)
train_sel['trans_count'] = train_sel['client_date'].apply(trans_count).astype(float)
train_sel['ltr_mean'] = train_sel['client_date'].apply(ltr_mean).astype(float)

test['date'] = test['DATA_TRANS'].apply(date)
test['client_date'] = test['CLIENT'].astype(str) + '_' + test['date'].astype(str)
test['trans_count'] = test['client_date'].apply(trans_count).astype(float)
test['ltr_mean'] = test['client_date'].apply(ltr_mean).astype(float)

CPU times: user 1min 52s, sys: 6.12 s, total: 1min 58s
Wall time: 1min 58s


In [10]:
%%time

# фичи про оценку размера бизнеса
# до 40 000 литров/месяц – массовый;
# от 40 000 до 150 000 литров/месяц – средний;
# от 150 000 литров/месяц – крупный;

train_sel['biz_estimated_ltr'] = train_sel['trans_count'] * train_sel['ltr_mean']
train_sel['biz_mass'] = train_sel['biz_estimated_ltr'].apply(lambda x: 1 if x <= 40000 else 0).astype(np.int16)
train_sel['biz_mid'] = train_sel['biz_estimated_ltr'].apply(lambda x: 1 if x > 40000 and x <= 150000 else 0).astype(np.int16)
train_sel['biz_big'] = train_sel['biz_estimated_ltr'].apply(lambda x: 1 if x > 150000 else 0).astype(np.int16)


test['biz_estimated_ltr'] = test['trans_count'] * test['ltr_mean']
test['biz_mass'] = test['biz_estimated_ltr'].apply(lambda x: 1 if x <= 40000 else 0).astype(np.int16)
test['biz_mid'] = test['biz_estimated_ltr'].apply(lambda x: 1 if x > 40000 and x <= 150000 else 0).astype(np.int16)
test['biz_big'] = test['biz_estimated_ltr'].apply(lambda x: 1 if x > 150000 else 0).astype(np.int16)

CPU times: user 48.4 s, sys: 4.38 s, total: 52.8 s
Wall time: 1min 10s


In [11]:
train_sel.sample(5)

,ID,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,COL_LITR,CENA_CLIENT,month,date,client_date,trans_count,ltr_mean,biz_estimated_ltr,biz_mass,biz_mid,biz_big
30352606,30352606,48189,11.12.2017 8:51:26,803,2,1,36.000000,38.349998,12.0,12.2017,48189_12.2017,157.0,113.183655,17769.833801,1,0,0
23169727,23169727,14331,06.10.2017 19:30:46,1746,19,1,30.000000,35.669998,10.0,10.2017,14331_10.2017,430.0,44.869381,19293.833809,1,0,0
17416208,17416208,9118,04.08.2017 19:55:27,736,18,2,55.869999,35.080002,8.0,08.2017,9118_08.2017,12.0,57.156818,685.881821,1,0,0
26619021,26619021,35880,09.11.2017 10:07:40,1631,21,2,20.000000,33.480000,11.0,11.2017,35880_11.2017,744.0,15.481934,11518.558594,1,0,0
16423096,16423096,15361,24.07.2017 11:00:45,974,6,2,40.000000,37.840000,7.0,07.2017,15361_07.2017,1871.0,33.823677,63284.099785,0,1,0


In [12]:
test.sample(5)

,ID,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,CENA_CLIENT,month,date,client_date,trans_count,ltr_mean,biz_estimated_ltr,biz_mass,biz_mid,biz_big
69510,69510,4609,21.02.2018 15:45:06,1306,4,1,39.259998,2.0,02.2018,4609_02.2018,12390.0,34.799137,4.311613e+05,0,0,1
458463,458463,24593,15.02.2018 13:14:17,1644,4,1,36.470001,2.0,02.2018,24593_02.2018,2970.0,269.675140,8.009352e+05,0,0,1
189070,189070,9909,15.01.2018 6:29:37,1711,17,2,38.910000,1.0,01.2018,9909_01.2018,236.0,34.596436,8.164759e+03,1,0,0
399302,399302,30916,21.02.2018 1:06:33,1041,2,1,37.669998,2.0,02.2018,30916_02.2018,12143.0,189.093323,2.296160e+06,0,0,1
480690,480690,26441,12.03.2018 23:05:50,1417,14,2,33.669998,3.0,03.2018,26441_03.2018,1511.0,22.747156,3.437095e+04,1,0,0


In [13]:
cols = ['CLIENT','AZS_NUMBER','REGION_AZS','VID_NP','CENA_CLIENT','month','trans_count','biz_estimated_ltr','biz_mass','biz_mid','biz_big']

In [14]:
X_train = train_sel[cols]
y = train_sel['COL_LITR']
X_test = test[cols]

del train, train_sel, test
gc.collect() 

38

In [15]:
import lightgbm

In [16]:
%%time

ctb = lightgbm.LGBMRegressor()
ctb.fit(X_train, y)

CPU times: user 10min 38s, sys: 6.18 s, total: 10min 45s
Wall time: 3min 8s


In [17]:
pd.DataFrame(X_train.columns, ctb.feature_importances_)

,0
252,CLIENT
477,AZS_NUMBER
379,REGION_AZS
157,VID_NP
196,CENA_CLIENT
22,month
690,trans_count
818,biz_estimated_ltr
5,biz_mass
4,biz_mid


In [18]:
y_test = ctb.predict(X_test)

In [19]:
y_test.shape

(1000000,)

In [20]:
X_test.shape

(1000000, 11)

In [21]:
sample_submission = pd.read_csv('datasets/gpn_b2b/sample_submission.csv')

In [23]:
sample_submission.tail()

,ID,COL_LITR
999995,999995,0.0
999996,999996,0.0
999997,999997,0.0
999998,999998,0.0
999999,999999,0.0


In [24]:
sample_submission['COL_LITR'] = y_test

In [25]:
sample_submission.tail()

,ID,COL_LITR
999995,999995,154.763564
999996,999996,185.467910
999997,999997,291.594827
999998,999998,41.274159
999999,999999,28.413629


In [26]:
sample_submission.to_csv('gpn_b2b_allm.csv.gz', compression='gzip',  index=False)